In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib as mpl

In [2]:
# import data
raw_df = pd.read_csv('f835051.csv')
# take out the golden training data (skews things dramatically)
raw_df = raw_df[raw_df['_golden'] == False]


In [3]:
# remove the entries for which there is no destination url 
nan_list = []

for endurl in raw_df['endurl']:
    nan_list.append(type(endurl) != type(float()))

    
reduced_raw_df = raw_df[nan_list]      
print reduced_raw_df.shape
print raw_df.shape

raw_df = reduced_raw_df

(9058, 26)
(9476, 26)


In [ ]:
# manipulate data for evaluation of workers

worker_mean = []
worker_var = []
worker_count = 0
worker_job_count = []
worker_list = []

for worker in raw_df.groupby('_worker_id'):
    worker_list.append(worker[0])
    worker_mean.append(worker[1]['please_make_your_relevancy_judgment'].mean())
    worker_var.append(worker[1]['please_make_your_relevancy_judgment'].var())
    worker_job_count.append(worker[1]['please_make_your_relevancy_judgment'].count())
    
    worker_count += 1

worker_count

worker_var_dict = dict(zip(worker_list, worker_var))
worker_count_dict = dict(zip(worker_list,worker_job_count))

In [34]:
indices = []
index = 0
for mean in worker_mean:
    if mean == 3:
        indices.append(index)
    index += 1
print indices
print worker_job_count[14]
worker_list[14]

# just to get an exmaple robot, this one is a robot: 22045386

[14, 77]
258


22045386

In [5]:
# part 1:
# remove the robots 

def remove_robots(job_count_threshold, job_variance_threshold):
    too_many_jobs = []
    for worker in worker_list:
        if worker_count_dict[worker] > job_count_threshold:
            too_many_jobs.append(worker)
    
    
    too_little_variance = []
    for worker in worker_list:
        if worker_var_dict[worker] < job_variance_threshold:
            too_little_variance.append(worker)
                  

    robots = set(too_many_jobs).intersection(set(too_little_variance))
    humans = set(worker_list).difference(robots)
    
    return humans 



In [6]:
humans = remove_robots(150, .2)
print "there are " + str((1 - (float(len(humans)) / len(worker_list))) * 100) + "% robots"
# even with pretty generous settings(more than 150 tasks with varaince less than .2), 10% of the workers as robots
human_scores = []

for worker_id in raw_df['_worker_id']:
    human_scores.append(worker_id in humans)
    
num_human_scores = sum(human_scores)
num_total_scores = raw_df['_worker_id'].count()
num_bot_scores = num_total_scores - num_human_scores

print str(round(float((num_bot_scores + 0.0)/num_total_scores + 0.0)* 100,2)) + "% of the scores are from robots"

there are 10.0% robots
22.44% of the scores are from robots


In [7]:
# reduce the scores data to just those by humans 
raw_df = raw_df[human_scores]

In [8]:
# manipulation of data for evaluation of scores 

raw_scores = []
means = []
variances = []
unit_ids = []
counts = []
scorer = []
no_two_var = []
modes = []
counter = 0


for score_set in raw_df.groupby('_unit_id')['please_make_your_relevancy_judgment']:
    raw_scores.append(score_set[1])
    unit_ids.append(score_set[0])
    means.append(score_set[1].mean())
    variances.append(score_set[1].var())
    no_two_var.append(score_set[1][score_set[1] != 2].var())
    counts.append(score_set[1].count())
    modes.append(score_set[1].mode()[0])
    counter += 1
    
    

In [9]:
def normalize(list_of_numbers):
    min_value = min(list_of_numbers)
    list_less_min = list_of_numbers - min_value
    max_value = max(list_less_min)
    normalized_values = list_less_min / max_value
    return normalized_values

rounded_normalized_means = normalize(means) > .5

In [10]:
new_modes = []
for mode in modes:
    if mode == 2:
        print "the most is two"
        new_modes.append(1)
    if mode == 1:
        new_modes.append(0)
    if mode == 3:
        new_modes.append(1)

sum((new_modes == rounded_normalized_means) == 0)

# there is an insight to be gained here
# when people are unsure of something is relevant
# if you say that it is relevant, that agrees with the rounded mean
# which goes to say that uncertainty about relvance is more about the evaluator
# than it is about the pair being evaluated 

the most is two
the most is two
the most is two
the most is two


0

In [11]:
aggregated_scores = new_modes
aggregated_df = pd.DataFrame()
aggregated_df['_unit_id'] = unit_ids
aggregated_df['score'] = new_modes
aggregated_df.to_csv('aggregated_scores.csv')

In [14]:
# put this into the database, so that i don't need to process it each time, 
#becuase it is very slow
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

dbname = 'urx'
username = 'noahburbank'

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

In [15]:
aggregated_df.to_sql('aggregated_scores', engine, if_exists = 'replace')